In [1]:
# Load model directly
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch



2024-01-30 16:40:27.771376: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-30 16:40:27.956514: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-30 16:40:27.993603: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-30 16:40:27.993641: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t12_35M_UR50D")
model = AutoModelForMaskedLM.from_pretrained("facebook/esm2_t12_35M_UR50D")

In [3]:
tokenizer.get_vocab()

{'<cls>': 0,
 '<pad>': 1,
 '<eos>': 2,
 '<unk>': 3,
 'L': 4,
 'A': 5,
 'G': 6,
 'V': 7,
 'S': 8,
 'E': 9,
 'R': 10,
 'T': 11,
 'I': 12,
 'D': 13,
 'P': 14,
 'K': 15,
 'Q': 16,
 'N': 17,
 'F': 18,
 'Y': 19,
 'M': 20,
 'H': 21,
 'W': 22,
 'C': 23,
 'X': 24,
 'B': 25,
 'U': 26,
 'Z': 27,
 'O': 28,
 '.': 29,
 '-': 30,
 '<null_1>': 31,
 '<mask>': 32}

In [4]:
model

EsmForMaskedLM(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 480, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 480, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-11): 12 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=480, out_features=480, bias=True)
              (key): Linear(in_features=480, out_features=480, bias=True)
              (value): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((480,), eps=1e-05, elementwise

In [7]:
df = pd.read_csv("/home/gvisona/Projects/SelfPeptides/processed_data/Immunogenicity/Processed_TCell_IEDB_beta_summed.csv")
df

,Peptide,HLA,Qualitative Measurement,Peptide length,Number of Subjects Tested,Number of Subjects Positive,Alpha,Beta
0,CYTWNQMNL,HLA-A24:02,Positive,9,84.0,60.0,61.0,25.0
1,GYDQIMPKK,HLA-A24:02,Positive,9,2.0,2.0,3.0,1.0
2,YLSGANLNL,HLA-A02:01,Positive,9,8.0,6.0,7.0,3.0
3,QYSWFVNGTF,HLA-A24:02,Positive,10,12.0,5.0,6.0,8.0
4,TYACFVSNL,HLA-A24:02,Positive,9,12.0,5.0,6.0,8.0
...,...,...,...,...,...,...,...,...
28295,RLLEQKVEL,HLA-A02:01,Positive-Low,9,12.0,3.0,4.0,10.0
28296,HLVDEAHCLRL,HLA-A02:01,Positive-Low,11,12.0,3.0,4.0,10.0
28297,QLVDIIEKV,HLA-A02:01,Positive,9,12.0,6.0,7.0,7.0
28298,ALSNLEVKL,HLA-A02:01,Positive,9,12.0,5.0,6.0,8.0


In [31]:
peptides = df["Peptide"].tolist()

In [8]:
batch = df["Peptide"].iloc[:8].tolist()
batch

['CYTWNQMNL',
 'GYDQIMPKK',
 'YLSGANLNL',
 'QYSWFVNGTF',
 'TYACFVSNL',
 'LMLGEFLKL',
 'AAAAAIFVI',
 'FLPSDFFPSV']

In [67]:
inputs = tokenizer(batch, return_tensors="pt", padding=True)
inputs

{'input_ids': tensor([[ 0, 23, 19, 11, 22, 17, 16, 20, 17,  4,  2,  1],
        [ 0,  6, 19, 13, 16, 12, 20, 14, 15, 15,  2,  1],
        [ 0, 19,  4,  8,  6,  5, 17,  4, 17,  4,  2,  1],
        [ 0, 16, 19,  8, 22, 18,  7, 17,  6, 11, 18,  2],
        [ 0, 11, 19,  5, 23, 18,  7,  8, 17,  4,  2,  1],
        [ 0,  4, 20,  4,  6,  9, 18,  4, 15,  4,  2,  1],
        [ 0,  5,  5,  5,  5,  5, 12, 18,  7, 12,  2,  1],
        [ 0, 18,  4, 14,  8, 13, 18, 18, 14,  8,  7,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [68]:
a = inputs["attention_mask"].sum(axis=1)
a

tensor([11, 11, 11, 12, 11, 11, 11, 12])

In [63]:
np.concatenate([np.random.choice(list(range(a[i])), size=2, replace=False) for i in range(len(a))])

array([10,  8,  5,  9,  0,  4,  7,  0,  4,  5,  2,  8,  8,  9,  9,  5])

In [69]:
from copy import deepcopy

In [70]:
def mask_tokenized_inputs(tokenized_dict, mlm_fraction=0.15, mask_token_id=32):
    out_dict = deepcopy(tokenized_dict.copy())
    lengths = tokenized_dict["attention_mask"].sum(axis=1)
    
    row_ixs = []
    col_ixs = []
    
    for i, l in enumerate(lengths):
        n_masked_tokens = int(mlm_fraction*l)
        row_ixs.extend([i]*n_masked_tokens)
        col_ixs.append(np.random.choice(list(range(l)), size=n_masked_tokens, replace=False))
    col_ixs = np.concatenate(col_ixs)
    row_ixs = np.array(row_ixs)
    out_dict["input_ids"][row_ixs, col_ixs] = mask_token_id
    return out_dict

In [73]:
masked_inputs = mask_tokenized_inputs(inputs, mask_token_id=tokenizer.mask_token_id)
masked_inputs

{'input_ids': tensor([[ 0, 23, 19, 11, 22, 32, 16, 20, 17,  4,  2,  1],
        [32,  6, 19, 13, 16, 12, 20, 14, 15, 15,  2,  1],
        [ 0, 19,  4,  8,  6,  5, 17,  4, 17,  4, 32,  1],
        [ 0, 16, 32,  8, 22, 18,  7, 17,  6, 11, 18,  2],
        [ 0, 11, 19,  5, 23, 18, 32,  8, 17,  4,  2,  1],
        [ 0,  4, 20,  4,  6, 32, 18,  4, 15,  4,  2,  1],
        [ 0,  5,  5,  5,  5,  5, 12, 18,  7, 32,  2,  1],
        [ 0, 18,  4, 14,  8, 13, 18, 18, 14,  8,  7, 32]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [72]:
inputs

{'input_ids': tensor([[ 0, 23, 19, 11, 22, 17, 16, 20, 17,  4,  2,  1],
        [ 0,  6, 19, 13, 16, 12, 20, 14, 15, 15,  2,  1],
        [ 0, 19,  4,  8,  6,  5, 17,  4, 17,  4,  2,  1],
        [ 0, 16, 19,  8, 22, 18,  7, 17,  6, 11, 18,  2],
        [ 0, 11, 19,  5, 23, 18,  7,  8, 17,  4,  2,  1],
        [ 0,  4, 20,  4,  6,  9, 18,  4, 15,  4,  2,  1],
        [ 0,  5,  5,  5,  5,  5, 12, 18,  7, 12,  2,  1],
        [ 0, 18,  4, 14,  8, 13, 18, 18, 14,  8,  7,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [17]:
inputs2 = inputs.copy()
inputs2

{'input_ids': tensor([[ 0, 23, 19, 11, 22, 17, 16, 20, 17,  4,  2,  1],
        [ 0,  6, 19, 13, 16, 12, 20, 14, 15, 15,  2,  1],
        [ 0, 19,  4,  8,  6,  5, 17,  4, 17,  4,  2,  1],
        [ 0, 16, 19,  8, 22, 18,  7, 17,  6, 11, 18,  2],
        [ 0, 11, 19,  5, 23, 18,  7,  8, 17,  4,  2,  1],
        [ 0,  4, 20,  4,  6,  9, 18,  4, 15,  4,  2,  1],
        [ 0,  5,  5,  5,  5,  5, 12, 18,  7, 12,  2,  1],
        [ 0, 18,  4, 14,  8, 13, 18, 18, 14,  8,  7,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [20]:
tokenizer.mask_token_id

32

In [19]:
for i in range(8):
    ix = np.random.randint(0, 12)
    inputs2["input_ids"][i,ix] = tokenizer.mask_token_id
inputs2

{'input_ids': tensor([[ 0, 23, 19, 11, 22, 17, 32, 20, 17,  4,  2,  1],
        [ 0,  6, 19, 13, 32, 12, 20, 14, 15, 15,  2,  1],
        [ 0, 19, 32,  8,  6,  5, 17,  4, 17,  4,  2,  1],
        [ 0, 16, 19,  8, 22, 18,  7, 32,  6, 11, 18,  2],
        [ 0, 11, 19,  5, 23, 18,  7,  8, 17, 32,  2,  1],
        [ 0,  4, 32,  4,  6,  9, 18,  4, 15,  4,  2,  1],
        [ 0,  5,  5, 32,  5,  5, 12, 18,  7, 12,  2,  1],
        [ 0, 18,  4, 14,  8, 13, 18, 18, 14, 32,  7,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [10]:
with torch.no_grad():
    logits = model(**inputs).logits
logits.shape

torch.Size([8, 12, 33])

In [21]:
tokenizer.mask_token_id

32

In [22]:
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)

labels = tokenizer(batch, return_tensors="pt",padding=True)["input_ids"]
# mask labels of non-<mask> tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)
labels

tensor([[-100, -100, -100, -100, -100, -100,   16, -100, -100, -100, -100, -100],
        [-100, -100, -100, -100,   16, -100, -100, -100, -100, -100, -100, -100],
        [-100, -100,    4, -100, -100, -100, -100, -100, -100, -100, -100, -100],
        [-100, -100, -100, -100, -100, -100, -100,   17, -100, -100, -100, -100],
        [-100, -100, -100, -100, -100, -100, -100, -100, -100,    4, -100, -100],
        [-100, -100,   20, -100, -100, -100, -100, -100, -100, -100, -100, -100],
        [-100, -100, -100,    5, -100, -100, -100, -100, -100, -100, -100, -100],
        [-100, -100, -100, -100, -100, -100, -100, -100, -100,    8, -100, -100]])

In [54]:
mask_token_index

tensor([6])

In [55]:
predicted_token_id

tensor([16])

In [29]:
outputs = model(**inputs, labels=labels, output_hidden_states=False)
outputs

MaskedLMOutput(loss=tensor(2.7584, grad_fn=<NllLossBackward0>), logits=tensor([[[ 13.8696,  -8.6695,  -0.3092,  ..., -15.3071, -15.1637,  -8.7102],
         [ -6.7248, -12.8521, -10.0629,  ..., -14.8430, -14.6449, -12.8491],
         [ -8.7960, -15.3293,  -8.4501,  ..., -15.6858, -15.6968, -15.3290],
         ...,
         [ -8.3384, -13.9196,  -8.1586,  ..., -15.3413, -15.3934, -13.9166],
         [ -1.8464,  -9.7893,  13.3795,  ..., -15.7441, -15.9714,  -9.7985],
         [ -9.0361, -14.4618,  -8.2976,  ..., -15.7336, -15.7147, -14.4684]],

        [[ 11.7932, -11.1072,  -2.3654,  ..., -15.4867, -15.2818, -11.1253],
         [ -8.6423, -15.4961,  -9.9237,  ..., -15.7949, -15.6557, -15.4870],
         [ -8.3243, -14.2958,  -8.0217,  ..., -15.6748, -15.6683, -14.2933],
         ...,
         [ -8.8828, -14.9908,  -8.6261,  ..., -15.6530, -15.6450, -14.9874],
         [ -2.7259, -10.9024,  12.5689,  ..., -15.8706, -16.1730, -10.8783],
         [ -7.9415, -13.5158,  -7.1330,  ..., -15.64

In [27]:
len(outputs.hidden_states)

13

In [28]:
outputs.hidden_states[-1].shape

torch.Size([8, 12, 480])

In [30]:
from transformers import DataCollatorForLanguageModeling

In [32]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [37]:
samples = [tokenizer(p, return_tensors="pt", padding=True) for p in peptides]
samples[:3]

[{'input_ids': tensor([[ 0, 23, 19, 11, 22, 17, 16, 20, 17,  4,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[ 0,  6, 19, 13, 16, 12, 20, 14, 15, 15,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[ 0, 19,  4,  8,  6,  5, 17,  4, 17,  4,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}]

In [38]:
for chunk in data_collator(samples)["peptide"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [41]:
df = pd.read_csv("/home/gvisona/Projects/SelfPeptides/processed_data/Immunogenicity/Processed_TCell_IEDB_beta_summed.csv")
ba_df = pd.read_csv("/home/gvisona/Projects/SelfPeptides/processed_data/Binding_Affinity/DHLAP_binding_affinity_data.csv")
dhlap_df = pd.read_csv("/home/gvisona/Projects/SelfPeptides/processed_data/Immunogenicity/DHLAP_immunogenicity_data.csv")

In [40]:
ba_df

,HLA,Peptide,Method,Measurement,Type,Affinity(nM),Label
0,HLA-B08:01,HMMVIFRLM,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,3439140.0,0
1,HLA-B08:01,NFLIKFLLI,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,1606610.0,0
2,HLA-A24:02,SSKYYIKNI,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,1449300.0,0
3,HLA-A11:01,KLYKNKSKQ,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,1360000.0,0
4,HLA-A02:01,AMEKSSKYY,purified MHC/competitive/fluorescence,half maximal inhibitory concentration (IC50),Negative,1166430.0,0
...,...,...,...,...,...,...,...
327173,HLA-A11:01,SVQMTLSK,purified MHC,qualitative binding,Positive,0.0,1
327174,HLA-A02:09,TLGIVCPI,cellular MHC/direct/fluorescence,qualitative binding,Positive,0.0,1
327175,HLA-A11:01,TTSPIPLK,purified MHC,qualitative binding,Positive,0.0,1
327176,HLA-A03:01,TVTTITVY,cellular MHC/direct/fluorescence,qualitative binding,Positive,0.0,1


In [43]:

la_df = pd.read_csv("/home/gvisona/Projects/SelfPeptides/processed_data/Binding_Affinity/HLA_Ligand_Atlas_processed.csv")
la_df

,Peptide,HLA,Label
0,KLFNEHIEAL,HLA-A11:01,0
1,KLFNEHIEAL,HLA-B40:02,0
2,KLFNEHIEAL,HLA-B58:01,0
3,KLFNEHIEAL,HLA-C02:02,0
4,KLFNEHIEAL,HLA-C07:01,0
...,...,...,...
724635,IAVDVTSAL,HLA-A11:01,0
724636,IAVDVTSAL,HLA-B15:01,0
724637,IAVDVTSAL,HLA-B35:01,1
724638,IAVDVTSAL,HLA-C03:03,1


In [45]:
peptides = la_df["Peptide"].unique().tolist()
peptides[:3]

['KLFNEHIEAL', 'SRVEAVYVL', 'GEDPRVSINV']

In [48]:
from torch.utils.data import Dataset, IterableDataset

In [51]:
dset = PeptidesDataset(peptides)
dset

In [52]:
class PeptidesDataset(Dataset):
    def __init__(self, peptides):
        super().__init__()
        self.peptides = peptides
        
    def __len__(self):
        return len(self.peptides)
    
    def __getitem__(self, ix):
        return self.peptides[ix]

In [53]:
dset[0]

'KLFNEHIEAL'